## 0. Setup

In [ ]:
# Opcional: ver versión actual de requests
import requests, sys
print("requests antes:", requests.__version__)
print(sys.version)

requests antes: 2.32.4
3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]


In [ ]:
# Mantener requests en 2.32.4 para que no se rompa google-colab
%%bash
cat > /tmp/constraints.txt << 'EOF'
requests==2.32.4
EOF

# Instalar paquetes principales respetando la constraint de requests
pip install -q -U -c /tmp/constraints.txt \
  langchain-community langchain-openai langchain-chroma chromadb pypdf tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
import requests
print("requests después:", requests.__version__)

requests después: 2.32.4


In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain.chains import RetrievalQA

# Obtener la API Key desde la variable de entorno (GitHub Actions -> Secrets)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Inicializar el modelo LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    temperature=0
)

# Inicializar embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    api_key=openai_api_key
)

In [ ]:
import os
from google.colab import files
import shutil, pathlib

# Carpeta destino
BASE_DIR = "/content/data/chroma"
os.makedirs(BASE_DIR, exist_ok=True)

print("Entorno listo: data chroma")
print("Colab detectado. Sube tus PDFs/DOCX aquí:")

# Botón de subida
uploaded = files.upload()

# Mover archivos al directorio destino
for name in uploaded.keys():
    shutil.move(name, os.path.join(BASE_DIR, pathlib.Path(name).name))

print(f"✅ {len(uploaded)} archivo(s) guardado(s) en {BASE_DIR}")

Entorno listo: data chroma
Colab detectado. Sube tus PDFs/DOCX aquí:


Saving Guía de problemas frecuentes con equipo de cómputo.pdf to Guía de problemas frecuentes con equipo de cómputo.pdf
Saving Guia Win10 Win11.pdf to Guia Win10 Win11.pdf
Saving PR-TI-SP-001 Procedimiento de Recepcion Diagnostico de Equipos de computo.pdf to PR-TI-SP-001 Procedimiento de Recepcion Diagnostico de Equipos de computo.pdf
✅ 3 archivo(s) guardado(s) en /content/data/chroma


## 1. Extracting files

In [ ]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredFileLoader

folder_path = BASE_DIR  # usa la carpeta de la celda 1

# Buscar archivos comunes (recursivo)
patterns = ["**/*.pdf", "**/*.txt", "**/*.md", "**/*.docx", "**/*.html", "**/*.htm"]
file_paths = []
for p in patterns:
    file_paths.extend(glob(os.path.join(folder_path, p), recursive=True))

if not file_paths:
    raise FileNotFoundError(
        f"No se encontraron archivos en {folder_path}. "
        "Sube documentos en la celda 1."
    )

docs = []
for path in file_paths:
    try:
        low = path.lower()
        if low.endswith(".pdf"):
            loader = PyPDFLoader(path)
        elif low.endswith(".txt") or low.endswith(".md"):
            loader = TextLoader(path)
        else:
            # Para DOCX/HTML/etc. requiere instalar "unstructured[standard]" y python-magic
            loader = UnstructuredFileLoader(path)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ Saltando {os.path.basename(path)}: {e}")

print(f"✅ Se cargaron {len(docs)} documentos desde {len(file_paths)} archivos")

✅ Se cargaron 28 documentos desde 3 archivos


## 2. Text Splitting into Chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
split_docs = text_splitter.split_documents(docs)
print(f"✅ Se crearon {len(split_docs)} chunks")

✅ Se crearon 49 chunks


## 3. Embedding

In [ ]:
import os

# Lee la API key (entorno o Colab Secrets)
openai_api_key = os.environ.get("OPENAI_API_KEY")
try:
    from google.colab import userdata
    openai_api_key = openai_api_key or userdata.get("OPENAI_API_KEY")
except Exception:
    pass

if not openai_api_key:
    raise ValueError(
        "No se encontró OPENAI_API_KEY. "
        "Configúrala como variable de entorno o en Colab (Runtime → Secrets)."
    )

from langchain_openai import OpenAIEmbeddings

# 1536 dimensiones garantizadas
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    api_key=openai_api_key
)

print("✅ Embeddings listos (dim=1536)")

✅ Embeddings listos (dim=1536)


## 4. Vector Stores

In [ ]:
from langchain_chroma import Chroma

PERSIST_DIR = "/content/chroma_db"

# Crear y guardar automáticamente en disco
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=PERSIST_DIR
)

print(f"✅ Vector store creado y guardado en: {PERSIST_DIR}")

✅ Vector store creado y guardado en: /content/chroma_db


## 5. Retriving from the Persistant Vector Datastore

In [ ]:
# 5) Retriving from the Persistent Vector Datastore
# ------------------------------------------------
# Reabre el vector store persistente de Chroma y prepara el retriever.

from langchain_chroma import Chroma
import os

# Verificación básica
if not os.path.isdir(PERSIST_DIR):
    raise FileNotFoundError(
        f"No existe el directorio de Chroma en {PERSIST_DIR}. "
        "Asegúrate de haber creado el store en la sección anterior."
    )

# Reabrir el store (usa los mismos embeddings con los que se indexó)
vectordb = Chroma(
    persist_directory=PERSIST_DIR,
    embedding_function=embeddings
)

# Crear el retriever (k = cantidad de chunks más similares a recuperar)
retriever = vectordb.as_retriever(search_kwargs={"k": 4})

print("✅ Store reabierto y retriever listo.")
print(f"📁 Persist directory: {PERSIST_DIR}")
print("🔎 Parámetros de búsqueda:", {"k": 4})

# (Opcional) Prueba rápida de recuperación sin LLM:
# ------------------------------------------------
# Descomenta para inspeccionar qué chunks devuelve el retriever.
# probe_query = "¿Qué información principal contienen estos documentos?"
# docs_preview = retriever.get_relevant_documents(probe_query)
# for i, d in enumerate(docs_preview, 1):
#     src = d.metadata.get("source", "desconocida")
#     print(f"\n--- Documento #{i} ---")
#     print("Fuente:", src)
#     print(d.page_content[:500], "...")

✅ Store reabierto y retriever listo.
📁 Persist directory: /content/chroma_db
🔎 Parámetros de búsqueda: {'k': 4}


## 6. Retrivers in Langchain

In [ ]:
# 6) Retrieval con LLM (QA) y múltiples consultas
# ------------------------------------------------

from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

# LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    temperature=0
)

# Chain de QA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# Lista de preguntas
queries = [
    "¿Qué información principal contienen estos documentos?",
    "¿La impresora atasca las hojas?"
]

# Ejecutar todas las consultas
for i, query in enumerate(queries, 1):
    result = qa.invoke({"query": query})
    print(f"\n🔎 Pregunta {i}: {query}")
    print("🧠 Respuesta:\n", result.get("result", result))


🔎 Pregunta 1: ¿Qué información principal contienen estos documentos?
🧠 Respuesta:
 Los documentos contienen información sobre procedimientos internos relacionados con la recepción, diagnóstico y reparación de equipos de cómputo. Incluyen secciones como "Inicio", "Nuevo Registro", "Editar Registro", "Cerrar Reparación", "Imprimir Reporte" y un anexo. También se menciona la revisión y emisión de un procedimiento específico con un código y la ubicación de la documentación en un módulo de control documental.

🔎 Pregunta 2: ¿La impresora atasca las hojas?
🧠 Respuesta:
 Sí, una de las fallas comunes es que el papel se traba siempre. La solución recomendada es llevar la impresora a mantenimiento al servicio técnico.
